# Sembradora 3000

This notebook presents an agent-based model that simulates the propagation of a disease through a network.
It demonstrates how to use the [agentpy](https://agentpy.readthedocs.io) package to create and visualize networks, use the interactive module, and perform different types of sensitivity analysis. 

In [ ]:
# Model design
import agentpy as ap
import random
import numpy as np
from collections import namedtuple, deque
from queue import PriorityQueue
from itertools import count
import math

# Visualization
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import HTML

# Machine Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## About the model

The agents of this model are people, which can be in one of the following three conditions: susceptible to the disease (S), infected (I), or recovered (R). The agents are connected to each other through a small-world network of peers. At every time-step, infected agents can infect their peers or recover from the disease based on random chance.

## Agente

In [ ]:
class CollectingTractor(ap.Agent):
    def setup(self, type = 1, pos = (0,0)):
        self.collected = 0
        self.targetIndex = 1
        self.path = []
        self.destroyed = False
        self.condition = True
        self.seeds = 0
        self.type = type
        self.pos = pos
        self.start = pos


    def heuristic(x1, y1, x2, y2):
        return abs(x1 - x2) + abs(y1 - y2)
    
    def a_star_search(self, start, end):
        #initialize the frontier using the initial state of the problem
        frontier = PriorityQueue()
        frontier.put(start, 0)
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0

        #run the search
        while not frontier.empty():
            current = frontier.get()

            if current == end:
                break

            #add to negihbors all the neighbors of the current cell
            neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0)]
            size = self.model.grid.shape[0]

            for nextTemp in neighbors:
                #check if the next cell is within the grid
                next = (current[0]+ nextTemp[0], current[1] + nextTemp[1])
                if next[0] < 0 or next[0] >= size or next[1] < 0 or next[1] >= size:
                    continue
                #check if the next cell is an obstacle
                if self.model.grid["occupied"][next] == 1:
                    continue

                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + abs(end[0] - next[0]) + abs(end[1] - next[1])
                    frontier.put(next, priority)
                    came_from[next] = current

        #reconstruct the path
        path = []
        current = end
        while current != start:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path
        
    def collect(self):
        #run an A* search to find the shortest path to the target
        if (self.type == 2):
            return

        if (self.destroyed):
            return


        start = (self.pos[0], self.pos[1])
        if (self.seeds > 0):
            end = self.targets[self.targetIndex]
        else:
            #iterate through the seeds and find the closest one using A*
            minDist = 100000
            for seed in self.model.p.seedsPositions:
                dist = len(self.a_star_search(start, seed))
                if (dist < minDist):
                    minDist = dist
                    end = seed

        self.model.np_grid[end] = 2
        if (len(self.path) == 0):
            self.path = self.a_star_search(start, end)

        
        if (len(self.path) == 0):
            next_pos = self.pos
        else:
            next_pos = self.path.pop(0)
            if not (next_pos in self.model.grid.empty.items):
                return        
        
        self.model.grid.move_to(self, next_pos)
        self.model.np_grid[self.pos] = 0
        self.model.np_grid[next_pos] = 1
        self.pos = next_pos

        if (self.pos in self.model.p.seedsPositions):
            self.seeds = self.capacity
            return
        #check if the target has been reached

        if self.pos == end:
            self.seeds -= 1
            self.model.Collected += 1
            self.targetIndex += 1
            if self.targetIndex == len(self.targets):
                self.model.np_grid[self.pos] = 0
                self.model.grid.remove_agents(self)
                self.destroyed = True
                self.condition = False
                self.type = 0

## Grid

#### Definir grid

In [ ]:
"""
1 is tractor
2 is obstacle
3 is target
4 is seeds
"""


def is_connected(grid, free_positions):
    """ Check if all free cells are connected using BFS """
    n = grid.shape[0]
    visited = set()
    queue = deque([free_positions.pop()])
    visited.add(queue[0])

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    connected_count = 1
    free_count = len(free_positions)

    while queue:
        x, y = queue.popleft()
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < n and 0 <= new_y < n and (new_x, new_y) in free_positions and (new_x, new_y) not in visited:
                queue.append((new_x, new_y))
                visited.add((new_x, new_y))
                connected_count += 1
        
                
    return connected_count > free_count

def generate_grid(model, n, obstacles_count):
    grid = ap.Grid(model, (n, n), track_empty=True)  # Create an agentpy Grid object
    grid.add_field("occupied", 0)  # Add a field to store obstacle information

    # Generate obstacle positions
    obstacle_positions = set()
    while len(obstacle_positions) < obstacles_count:
        x, y = random.randint(0, n-1), random.randint(0, n-1)
        if (x, y) not in obstacle_positions and (x,y) not in model.p.seedsPositions:
            obstacle_positions.add((x, y))

    # Mark grid cells as obstacles
    for pos in obstacle_positions:
        grid["occupied"][pos] = 1

    # Identify free positions
    free_positions = set()
    for pos in grid.all:
        if grid["occupied"][pos] != 1:
            free_positions.add(pos)

    # Check if the free cells are connected
    final_obstacles = set()
    while not is_connected(grid, free_positions):
        final_obstacles.clear()
        grid = ap.Grid(model, (n, n), track_empty=True)
        grid.add_field("occupied", 0)
        obstacle_positions = set()
        while len(obstacle_positions) < obstacles_count:
            x, y = random.randint(0, n-1), random.randint(0, n-1)
            if (x, y) not in obstacle_positions and (x,y) not in model.p.seedsPositions:
                obstacle_positions.add((x, y))
        
        free_positions = set()
        for pos in grid.all:
            if grid["occupied"][pos] == 0:
                free_positions.add(pos)
        final_obstacles = obstacle_positions

    for pos in obstacle_positions:
        grid["occupied"][pos] = 1
        model.np_grid[pos] = 3
    #add an agent to each obstacle position with type 2
    #Make an agentlist ap.agentlist
    #Add the agent to the grid
    #Add the agent to the agentlist
    agentlist = ap.AgentList(model, len(obstacle_positions))
    agentlist.type = 2
        
    grid.add_agents(agentlist, obstacle_positions)
    


    model.grid = grid

#### Ambiente

In [ ]:
class TractorModel(ap.Model):
    def setup(self):
        self.Collected = 0
        #numpy array of size grid_size x grid_size
        self.np_grid = np.zeros((self.p.grid_size, self.p.grid_size))
        generate_grid(self, self.p.grid_size, self.p.obstacles_count)
        #Create p.number_of_tractors tractors
        self.agents = ap.AgentList(self, self.p.number_of_tractors, CollectingTractor)
        self.agents.capacity = self.p.capacity
        self.agents.seeds = self.p.starting_seeds
        #Unique coords
        coordsUsed = set()
        #Set the targets for each tractor, checking that they are not obstacles
        for tractor in self.agents:
            targets = []
            for i in range(self.p.number_of_targets + 1):
                x, y = random.randint(0, self.p.grid_size-1), random.randint(0, self.p.grid_size-1)
                while self.grid["occupied"][(x, y)] == 1 or (x, y) in coordsUsed:
                    x, y = random.randint(0, self.p.grid_size-1), random.randint(0, self.p.grid_size-1)
                targets.append((x, y))
                coordsUsed.add((x, y))
            tractor.targets = targets
            tractor.pos = targets[0]
            self.np_grid[tractor.pos] = 1
        
        self.grid.add_agents(self.agents, [tractor.pos for tractor in self.agents])
        
    def update(self):
        self.record('Collected', sum([tractor.collected for tractor in self.agents]))

        
    def step(self):
            self.agents.collect()
            #Assign 4 to the seed positions
            for seed in self.p.seedsPositions:
                self.np_grid[seed] = 4
        
    def end(self):
        self.report('Total targets', self.p.number_of_targets * self.p.number_of_tractors)
        #time to collect all targets

## Parameters

In [ ]:


tractorParameters = {
    'grid_size': 10,
    'obstacles_count': 10,
    'number_of_tractors': 4,
    'number_of_targets': 8,
    'steps': 100,
    'seedsPositions': [(0, 0)],
    'capacity': 2,
    'starting_seeds': 2
}

model = TractorModel(tractorParameters)
results = model.run(steps=100)
'''model = VirusModel(parameters)
results = model.run() '''

## Machine Learning

In [ ]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = model.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = model.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

## Analyzing results

In [ ]:
''' 
ESTO NO JALA, PERO SI ALGUIEN LO QUIERE ARREGLAR, DESE
def tractor_plot(data, ax):
    x = data.index.get_level_values('t')
    y = data['Collected']
    ax.plot(x, y, label='Collected targets')
    ax.legend()
    ax.set_xlim(0, max(1, len(x)-1))
    ax.set_ylim(0, 25)
    ax.set_xlabel("Time steps")
    ax.set_ylabel("Number of collected targets")

fig, ax = plt.subplots()
tractor_plot(results.variables.TractorModel, ax)
'''

## Creating an animation

In [ ]:
"""
0 is empty
1 is tractor
2 is obstacle
3 is target
4 is seeds
"""

# Load images
tractor_img = mpimg.imread('tractor.png')
obstacle_img = mpimg.imread('obstacle.png')
target_img = mpimg.imread('target.png')
seeds_img = mpimg.imread('seeds.png')

def animation_plot(model, ax):
    # Clear the axis to avoid over-plotting
    ax.clear()

    # Plot the grid using images
    for (x, y), value in np.ndenumerate(model.np_grid):
        if value == 1:  # Tractor
            ax.imshow(tractor_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 3:  # Obstacle
            ax.imshow(obstacle_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 2:  # Target
            ax.imshow(target_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 4:  # Seeds
            ax.imshow(seeds_img, extent=[y, y+1, x, x+1], aspect='auto')

    # Add text for each tractor displaying the number of seeds it has
    for agent in model.agents:
        if agent.destroyed:
            continue
        ax.text(agent.pos[1] + 0.5, agent.pos[0] + 0.5, str(agent.seeds),
                color='black', fontsize=12, ha='center', va='center', weight='bold')

    # Fix axis limits based on the grid dimensions
    ax.set_xlim([0, model.np_grid.shape[1]])
    ax.set_ylim([0, model.np_grid.shape[0]])

    # Set aspect ratio to 'equal' to prevent image stretching
    ax.set_aspect('equal')

    # Set the title for the plot
    ax.set_title(f"Tractor model \n Time-step: {model.t}, Collected: {model.Collected}")

# Example usage
fig, ax = plt.subplots()
model = TractorModel(tractorParameters)
animation = ap.animate(model, fig, ax, animation_plot)
animation.save('simulacionTractores.gif')


In [ ]:
#Display the animation
HTML(animation.to_html5_video())